In [1]:
import duckdb
import polars as pl

In [2]:
# 1. Use DuckDB to query the files without loading them into memory
con = duckdb.connect()

query = """
SELECT 
    t.user_id,
    u.name,
    t.amount
FROM 
    'transactions.parquet' t
JOIN 
    'users.parquet' u 
ON 
    t.user_id = u.user_id
WHERE 
    u.country = 'Japan'
    AND SUBSTR(t.timestamp, 1, 4) = '2024'
"""

# Run query and get the result as a Polars DataFrame
df_duck = con.execute(query).pl()

In [3]:
df_duck

user_id,name,amount
i64,str,f64
32,"""User_32""",923.17
62,"""User_62""",1348.27
30,"""User_30""",295.31
62,"""User_62""",1184.72
27,"""User_27""",237.59
…,…,…
30,"""User_30""",1112.08
78,"""User_78""",600.3
72,"""User_72""",222.53


In [4]:
type(df_duck)

polars.dataframe.frame.DataFrame

In [5]:
# 2. Use Polars for fast aggregation
df_polars = (
    df_duck
    .group_by("name")
    .agg(pl.col("amount").sum().alias("total_spent"))
    .sort("total_spent", descending=True)
)

print(df_polars)

shape: (33, 2)
┌─────────┬─────────────┐
│ name    ┆ total_spent │
│ ---     ┆ ---         │
│ str     ┆ f64         │
╞═════════╪═════════════╡
│ User_33 ┆ 7291.23     │
│ User_38 ┆ 6485.24     │
│ User_69 ┆ 6308.2      │
│ User_32 ┆ 6039.16     │
│ User_62 ┆ 4820.92     │
│ …       ┆ …           │
│ User_16 ┆ 1741.61     │
│ User_15 ┆ 1634.05     │
│ User_65 ┆ 1114.69     │
│ User_43 ┆ 862.54      │
│ User_20 ┆ 585.29      │
└─────────┴─────────────┘
